In [170]:
from numpy import mean,std
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

#import data
file_path = "~\CS634_Data_Mining\FinalProject\parkinsons.data"
# above .data file is comma delimited
parkinsons_data = pd.read_csv(file_path, delimiter=",")

#X data definition
del parkinsons_data['name']
data = parkinsons_data.iloc[1:195,0:17]
X = parkinsons_data.iloc[1:195,0:16]

#convert to np array
X_df = X
X = X.to_numpy()
X_list = X.tolist()

#y data definition
y = parkinsons_data.iloc[1:195,16]

#convert to np array
y_df = y
y = y.to_numpy()

In [222]:
#Support vector classifier
#classifier = make_pipeline(StandardScaler(), SVC(gamma='auto'))

#split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

classifier = SVC(kernel='poly',degree = 10, C=1)
#classifier.fit(X_train,y_train)
#classifier.predict(X_test)
#classifier.score(X_test,y_test)



#cross validation where k=10
cv_SVC = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
scores = cross_val_score(classifier, X, y, scoring='accuracy', cv=cv_SVC, n_jobs=-1)
#print(scores)

# report performance
print('Accuracy using scikit-learn CV: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy using scikit-learn CV: 0.852 (0.072)


In [223]:
#random forest

forest_classifier = RandomForestClassifier(max_depth=None, random_state=0,max_features=15)
#forest_classifier.fit(X_train,y_train)
#forest_classifier.predict(X_test)
#forest_classifier.score(X_test,y_test)

cv_forest = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
forest_scores = cross_val_score(forest_classifier, X, y, scoring='accuracy', cv=cv_forest, n_jobs=-1)

# report performance
print('Accuracy using scikit-learn CV: %.3f (%.3f)' % (mean(forest_scores), std(forest_scores)))

Accuracy using scikit-learn CV: 0.882 (0.069)


In [245]:
import numpy as np
import random

def cross_validation_split(dataset, folds):
        dataset_split = []
        df_copy = dataset
        fold_size = int(df_copy.shape[0] / folds)
        
        # for loop to save each fold
        for i in range(folds):
            fold = []
            # while loop to add elements to the folds
            while len(fold) < fold_size:
                # select a random element
                r = random.randrange(df_copy.shape[0])
                # determine the index of this element 
                index = df_copy.index[r]
                # save the randomly selected line 
                fold.append(df_copy.loc[index].values.tolist())
                # delete the randomly selected line from
                # dataframe not to select again
                df_copy = df_copy.drop(index)
            # save the fold     
            dataset_split.append(np.asarray(fold))
        
        #dataset_split = pd.DataFrame(dataset_split[0])
        return dataset_split 


def tenFoldCV(dataset,f,k,model):
    #do folds
    data_split = cross_validation_split(dataset,k)
    results = []
    for i in range(f):
        r = list(range(f))
        r.pop(i)
        for j in r:
            if j == r[0]:
                cv = data_split[j]
            else:
                cv = np.concatenate((cv,data_split[j]),axis=0)
        
        #assign correct labels
        labels = []
        for k in range(len(cv)):
            val = cv[k][16]
            labels.append(val)
        #print(labels)
        if model == 'SVC':
            clf = SVC(kernel='poly',degree = 10, C=1)
            clf.fit(cv[:,0:16],labels)
            y_pred = clf.predict(data_split[i][:,0:16])
            
            #metrics
            y_true = data_split[i][:,16]
            TP = 0
            FP = 0
            TN = 0
            FN = 0
            correct = 0
            for m in range(len(y_pred)):
                if y_pred[m] == y_true[m] and y_pred[m] == 1:
                    TP += 1
                    correct += 1
                elif y_pred[m] == y_true[m] and y_pred[m] == 0:
                    TN += 1
                    correct += 1
                elif y_pred[m] != y_true[m] and y_pred[m] == 1:
                    FP += 1
                elif y_pred[m] != y_true[m] and y_pred[m] == 0:
                    FN += 1
            #print("Run",i+1,"Performance:")
            #print("Total correct: ",correct,"/",len(y_pred))
            performance = correct/len(y_pred)
            #print("Model performance: ",performance)
            #print("True Positives: ",TP)
            #print("False Positives: ",FP)
            #print("True Negatives: ",TN)
            #print("False Negatives: ",FN)
            #print('#'*75)
            run_results = [performance,TP,FP,TN,FN]
            results.append(run_results)
    
        elif model == 'RandomForest':
            clf = RandomForestClassifier(max_depth=None, random_state=0,max_features=15)
            clf.fit(cv[:,0:16],labels)
            y_pred = clf.predict(data_split[i][:,0:16])
            
            #metrics
            y_true = data_split[i][:,16]
            TP = 0
            FP = 0
            TN = 0
            FN = 0
            correct = 0
            for m in range(len(y_pred)):
                if y_pred[m] == y_true[m] and y_pred[m] == 1:
                    TP += 1
                    correct += 1
                elif y_pred[m] == y_true[m] and y_pred[m] == 0:
                    TN += 1
                    correct += 1
                elif y_pred[m] != y_true[m] and y_pred[m] == 1:
                    FP += 1
                elif y_pred[m] != y_true[m] and y_pred[m] == 0:
                    FN += 1
            #print("Run",i+1,"Performance:")
            #print("Total correct: ",correct,"/",len(y_pred))
            performance = correct/len(y_pred)
            #print("Model performance: ",performance)
            #print("True Positives: ",TP)
            #print("False Positives: ",FP)
            #print("True Negatives: ",TN)
            #print("False Negatives: ",FN)
            #print('#'*75)
            run_results = [performance,TP,FP,TN,FN]
            results.append(run_results)
            
            
            
            
    #mean performance
    total_perf = 0
    for index in range(len(results)):
        total_perf += results[index][0]
    mean_perf = total_perf / 10

    #find standard deviation between performances
    numerators = []
    for index2 in range(len(results)):
        numerator_ind = (results[index2][0] - mean_perf)**2
        numerators.append(numerator_ind)
    sum_num = sum(numerators)
    std_dev = np.sqrt(sum_num/len(results))
    
    
    print("Mean Performance:",mean_perf,"\nStandard Deviation: ",std_dev,"\n")
    print("#"*75)
    return total_perf

In [248]:
print("1. Support Vector Classification")
print("2. Random Forest Classification")
print("#"*75)
choice = int(input("Select which model you would like to run (1 or 2): "))
try:
    if choice == 1:
        performances = []
        for i in range(10):
            print("Run",i," results:")
            run = tenFoldCV(data,10,10,'SVC')
            performances.append(run)
        num = 0
        for index in range(len(performances)):
            num += performances[index]
        mean_num = num / len(performances*10)
        print("Mean Performance after 10 Repeats of 10fold CV: ", mean_num)
        
    elif choice == 2:
        performances = []
        for i in range(10):
            print("Run",i," results:")
            run = tenFoldCV(data,10,10,'RandomForest')
            performances.append(run)
        num = 0
        for index in range(len(performances)):
            num += performances[index]
        mean_num = num / len(performances*10)
        print("Mean Performance after 10 Repeats of 10fold CV: ", mean_num)
    else:
        print("Not a valid choice")
except:
    print("Not a valid choice")

1. Support Vector Classification
2. Random Forest Classification
###########################################################################
Select which model you would like to run (1 or 2): 1
Run 0  results:
Mean Performance: 0.8473684210526315 
Standard Deviation:  0.07608859102526823 

###########################################################################
Run 1  results:
Mean Performance: 0.8315789473684211 
Standard Deviation:  0.10473551969543367 

###########################################################################
Run 2  results:
Mean Performance: 0.8421052631578947 
Standard Deviation:  0.07443229275647868 

###########################################################################
Run 3  results:
Mean Performance: 0.8157894736842104 
Standard Deviation:  0.048523918196278354 

###########################################################################
Run 4  results:
Mean Performance: 0.8473684210526317 
Standard Deviation:  0.05494898162584499 

################